In [116]:
%%shell
jupyter nbconvert --to html "/content/project04.ipynb"

[NbConvertApp] Converting notebook /content/project04.ipynb to html
[NbConvertApp] Writing 387060 bytes to /content/project04.html


In [1]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 747kB 4.4MB/s 


In [80]:
import pandas as pd
import numpy as np
import re

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from collections import Counter

# Read NE.txt

In [81]:
path = "/content/drive/MyDrive/2020-2021 401/CS 445/Projects/CS 445 Project 4/NE.txt"

dataFile = open(path, "r")
dataLines = dataFile.readlines()

data = []
rawData = []

for line in dataLines:
    temp = line.replace("  ", " ")
    temp = temp.replace(" '", "'")
    temp = temp.replace("\n","")
    # temp2 = re.sub("\'","'", temp)
    rawData.append(temp)
    
    temp = re.sub(r'<.*?>',"", temp)
    data.append(temp)

In [82]:
data[0]

"Müzik Şenliği'ne hazırlanın POZİTİF ve Açık Radyo işbirliğiyle düzenlenecek olan İstanbul Müzik Şenliği 2 , müzikseverlere Aralık ayında merhaba demeye hazırlanıyor "

In [83]:
rawData[0]

'Müzik Şenliği\'ne hazırlanın <b_enamex TYPE="ORGANIZATION">POZİTİF ve Açık Radyo<e_enamex> işbirliğiyle düzenlenecek olan <b_enamex TYPE="LOCATION">İstanbul<e_enamex> Müzik Şenliği 2 , müzikseverlere Aralık ayında merhaba demeye hazırlanıyor '

In [84]:
paths = [ 
         "/content/drive/MyDrive/2020-2021 401/CS 445/Projects/CS 445 Project 4/Organization Database/organization_top_companies.txt",
        "/content/drive/MyDrive/2020-2021 401/CS 445/Projects/CS 445 Project 4/Organization Database/organization_turkey_top.txt",
        "/content/drive/MyDrive/2020-2021 401/CS 445/Projects/CS 445 Project 4/Organization Database/organization_turkish_banks.txt",
        "/content/drive/MyDrive/2020-2021 401/CS 445/Projects/CS 445 Project 4/Organization Database/organization_turkish_kurum.txt",
]

orgData = []

for path in paths:
    orgFile = open(path, "r")
    orgData += orgFile.read().splitlines()
    orgFile.close()

for line in orgData:
    if line == "":
        orgData.remove("")

In [85]:
orgData[-10:]

['Sosyal Güvenlik Kurumu',
 'Kadının Statüsü Genel Müdürlüğü',
 'Aile ve Sosyal Araştırma Genel Müdürlüğü',
 'Türkiye İş Kurumu',
 'Özel Çevre Koruma Kurumu',
 'İller Bankası Genel Müdürlüğü',
 'Gençlik ve Spor Genel Müdürlüğü',
 'Eximbank Genel Müdürlüğü',
 'Tarım ve Kırsal Kalkınmayı Destekleme Kurumu Başkanlığı',
 'TRT Genel Müdürlüğü']

# Read NE.ma.txt and Feature Mapping

In [86]:
def printResults(res):
    for key, value in res.items():
        print("Results for:", key)
        print("\tFlat_Accuracy:", value[0])
        print("\tFlat_F1_Score:", value[1])
        print("\tFlat_Precision:", value[2])
        print("\tFlat_Recall:", value[3])
             

In [91]:
def mapFeatures(word, ma, BOS, counter):
    
    stemIndex = ma.find("+")
    stem = ma[:stemIndex]

    ma = ma[stemIndex+1:]
    
    if counter > 0:
        if "DB" in ma:
            POSINDEX = ma.find("DB")
            POS = ma[POSINDEX+3:ma.find("+", POSINDEX+3)]
        else:
            POS = ma[:ma.find("+")]

    if counter > 1:
        PROP = False
        if "Prop" in ma:
            PROP = True

    if counter > 2:
        if "Nom" in ma:
            NounCase = "Nom"
        elif "Acc" in ma:
            NounCase = "Acc"
        elif "Dat" in ma:
            NounCase = "Dat"
        elif "Abl" in ma:
            NounCase = "Abl"
        elif "Loc" in ma:
            NounCase = "Loc"
        elif "Gen" in ma:
            NounCase = "Gen"
        elif "Ins" in ma:
            NounCase = "Ins"
        elif "Equ" in ma:
            NounCase = "Equ"
        else:
            NounCase = False

    if counter > 3:
        OCS = "LC"
        if word.isupper() and not BOS:
            OCS = "UC"

    if counter > 4:
        if "DB" in ma:
            POSINDEX = ma.find("DB")
            INFIndex = ma.find("+", POSINDEX+3)
            INF = ma[INFIndex+1:]
        else:
            INFIndex = ma.find("+")
            INF = ma[INFIndex+1:]

    if counter > 5:
        inORG = False
        for line in orgData:
            if word.upper() in line.upper():
                inORG = True

    if counter == 0:
        features = {
            'Stem': stem,       # stem
            'BOS': BOS,         # start of sentence
        }
    elif counter == 1:
        features = {
            'Stem': stem,       # stem
            'POS': POS,         # part-of-speech
            'BOS': BOS,         # start of sentence
        }
    elif counter == 2:
        features = {
            'Stem': stem,       # stem
            'POS': POS,         # part-of-speech
            'PROP': PROP,       # proper noun
            'BOS': BOS,         # start of sentence
        }
    elif counter == 3:
        features = {
            'Stem': stem,       # stem
            'POS': POS,         # part-of-speech
            'PROP': PROP,       # proper noun
            'NCS': NounCase,    # noun-case
            'BOS': BOS,         # start of sentence
        }
    
    elif counter == 4:
        features = {
            'Stem': stem,       # stem
            'POS': POS,         # part-of-speech
            'PROP': PROP,       # proper noun
            'NCS': NounCase,    # noun-case
            'OCS': OCS,
            'BOS': BOS,         # start of sentence
        }
    
    elif counter == 5:
        features = {
            'Stem': stem,       # stem
            'POS': POS,         # part-of-speech
            'PROP': PROP,       # proper noun
            'NCS': NounCase,    # noun-case
            'OCS': OCS,
            'INF': INF,
            'BOS': BOS,         # start of sentence
        }

    elif counter > 5:
        features = {
            'Stem': stem,       # stem
            'POS': POS,         # part-of-speech
            'PROP': PROP,       # proper noun
            'NCS': NounCase,    # noun-case
            'OCS': OCS,
            'INF': INF,
            'BOS': BOS,         # start of sentence
            'INORG': inORG,     # is in datalist
        }

    return features

In [119]:
for i in range(7):
    # ('Müzik', 'müzik+Noun+A3sg+Pnon+Nom'),
    print( mapFeatures("Müzik", 'müzik+Noun+A3sg+Pnon+Nom', True, i), "\n")

# # ('hazırlanın', 'hazırla+Verb^DB+Verb+Reflex+Pos+Imp+A2pl') hazırlanın
# print( mapFeatures("hazırlanın", 'hazırla+Verb^DB+Verb+Reflex+Pos+Imp+A2pl', False, 5), "\n")

# # 'olan', 'ol+Verb+Pos^DB+Adj+PresPart') olan
# print( mapFeatures("olan", 'ol+Verb+Pos^DB+Adj+PresPart', False, 4))

{'Stem': 'müzik', 'BOS': True} 

{'Stem': 'müzik', 'POS': 'Noun', 'BOS': True} 

{'Stem': 'müzik', 'POS': 'Noun', 'PROP': False, 'BOS': True} 

{'Stem': 'müzik', 'POS': 'Noun', 'PROP': False, 'NCS': 'Nom', 'BOS': True} 

{'Stem': 'müzik', 'POS': 'Noun', 'PROP': False, 'NCS': 'Nom', 'OCS': 'LC', 'BOS': True} 

{'Stem': 'müzik', 'POS': 'Noun', 'PROP': False, 'NCS': 'Nom', 'OCS': 'LC', 'INF': 'A3sg+Pnon+Nom', 'BOS': True} 

{'Stem': 'müzik', 'POS': 'Noun', 'PROP': False, 'NCS': 'Nom', 'OCS': 'LC', 'INF': 'A3sg+Pnon+Nom', 'BOS': True, 'INORG': False} 



In [95]:
print( mapFeatures("Teknik", 'teknik+Noun+Prop+A3sg+Pnon+Nom', False, 6) )

{'Stem': 'teknik', 'POS': 'Noun', 'PROP': True, 'NCS': 'Nom', 'OCS': 'LC', 'INF': 'Prop+A3sg+Pnon+Nom', 'BOS': False, 'INORG': False}


In [96]:
def maParser(line):
    firstSpace = line.find(" ")
    secondSpace = line.find(" ", firstSpace+1)
    lineNum = int(line[:firstSpace])
    word = line[firstSpace+1:secondSpace]
    ma = line[secondSpace+1:]

    return lineNum, word, ma

In [97]:
def getLabel(line, word):

    if line.find("<") == -1:
        return "O"

    while line.find("<") != -1:
        start_1 = line.find("<")
        end_1 = line.find(">")
        temp = line[start_1:end_1-1]
        typeIndex = temp.find("TYPE")

        start_2 = line.find("<", end_1+1)
        
        labelList = line[end_1+1:start_2].split()

        if word in labelList:
            if labelList[0] == word:
                labelType = temp[typeIndex+6:]
                labelType = "B-" + labelType
                break
            else:
                labelType = temp[typeIndex+6:]
                labelType = "I-" + labelType
                break
        else:
            labelType = "O"
        
        end_2 = line.find(">", start_2+1)
        line = line[end_2+3:]

    return (labelType)

In [98]:
line = 'Müzik Şenliği\'ne hazırlanın  <b_enamex TYPE="ORGANIZATION">POZİTİF ve Açık Radyo<e_enamex>  işbirliğiyle düzenlenecek olan  <b_enamex TYPE="LOCATION">İstanbul<e_enamex>'
print( getLabel( line, "İstanbul") )
print( getLabel( line, "POZİTİF") )
print( getLabel( line, "Radyo") )

B-LOCATION
B-ORGANIZATION
I-ORGANIZATION


# Run All

In [110]:
%%time
includedFeatures = [ "Stem", "POS", "PROP", "NCS", "OCS", "INF", "INORG"]

path = "/content/drive/MyDrive/2020-2021 401/CS 445/Projects/CS 445 Project 4/NE.ma.txt"

maFile = open(path, "r")
maLines = maFile.read().splitlines()

results = {}
avgResults = {}

for counter in range(7):
    IncFeatures = ""
    for index in range(counter+1):
        IncFeatures += includedFeatures[index] + " "

    IncFeatures += "BOS"

    print("***\nStarted for counter:", counter, ":", IncFeatures)

    maData = []
    i = 0
    for line in maLines:
        lineNum, word, ma = maParser(line)
        if lineNum > len(maData):
            maData.append( [line] )
        else:
            maData[lineNum-1].append(line)

    X = []
    y = []

    prevLine = 1
    index = 0
    for line in maData:
        temp_X = []
        temp_y = []
        BOS = True
        for word in line:
            lineNum, word, ma = maParser(word)
            temp_X.append( mapFeatures(word, ma, BOS, counter) )
            temp_y.append( getLabel(rawData[lineNum-1], word))
            BOS = False
        X.append(temp_X)
        y.append(temp_y)

    X_fold0 = []
    X_fold1 = []
    X_fold2 = []
    X_fold3 = []
    X_fold4 = []

    y_fold0 = []
    y_fold1 = []
    y_fold2 = []
    y_fold3 = []
    y_fold4 = []

    X_folds = [ X_fold0, X_fold1, X_fold2, X_fold3, X_fold4 ]
    y_folds = [ y_fold0, y_fold1, y_fold2, y_fold3, y_fold4 ]

    for index, sentence in enumerate(X):
        X_folds[index % 5].append(sentence)
    
    for index, label in enumerate(y):
        y_folds[index % 5].append(label)

    print("\tFolding is done.")

    foldAverages = [0, 0, 0, 0]

    for testFold in range(5):
        print("\t\tTestfold:", testFold)
        loopName = str(counter) + "_" + str(testFold)
        
        X_train = []
        y_train = []

        for index, fold in enumerate(X_folds):
            if index != testFold:
                X_train += fold

        for index, labels in enumerate(y_folds):
            if index != testFold:
                y_train += labels

        X_test = X_folds[testFold]
        y_test = y_folds[testFold]

        crf = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=0.1,
            c2=0.1,
            max_iterations=100,
            all_possible_transitions=True, )
        
        crf.fit(X_train, y_train)
        
        labels = list(crf.classes_)
        labels.remove("O")
        sorted_labels = sorted( labels, key=lambda name: (name[1:], name[0]))

        predictions = crf.predict(X_test)

        acc = metrics.flat_accuracy_score(y_test, predictions)
        f1 = metrics.flat_f1_score(y_test, predictions, average="weighted", labels=sorted_labels)
        precision = metrics.flat_precision_score(y_test, predictions, average="weighted", labels=sorted_labels)
        recall = metrics.flat_recall_score(y_test, predictions, average="weighted", labels=sorted_labels)

        foldAverages[0] += acc
        foldAverages[1] += f1
        foldAverages[2] += precision
        foldAverages[3] += recall
        results[loopName] = [ acc, f1, precision, recall]

        # print( "TestFold is:", testFold ," --> Flat f1 score:", metrics.flat_f1_score(y_test, predictions, average="weighted") )
        print( metrics.flat_classification_report( y_test, predictions, labels=sorted_labels, digits=3) )

    avgResults[IncFeatures] = [
                               foldAverages[0] / 5,
                               foldAverages[1] / 5,
                               foldAverages[2] / 5,
                               foldAverages[3] / 5,
    ]

    print("***\n")




***
Started for counter: 0 : Stem BOS
	Folding is done.
		Testfold: 0
                precision    recall  f1-score   support

    B-LOCATION      0.589     0.476     0.526       492
    I-LOCATION      0.612     0.448     0.517        67
B-ORGANIZATION      0.706     0.460     0.557       507
I-ORGANIZATION      0.638     0.387     0.482       341
      B-PERSON      0.857     0.633     0.728       864
      I-PERSON      0.726     0.524     0.609       334

     micro avg      0.726     0.519     0.605      2605
     macro avg      0.688     0.488     0.570      2605
  weighted avg      0.725     0.519     0.604      2605

		Testfold: 1
                precision    recall  f1-score   support

    B-LOCATION      0.643     0.497     0.561       465
    I-LOCATION      0.656     0.375     0.477        56
B-ORGANIZATION      0.715     0.475     0.571       448
I-ORGANIZATION      0.676     0.397     0.500       320
      B-PERSON      0.844     0.609     0.708       888
      I-PERSON  

In [111]:
printResults(avgResults)

Results for: Stem BOS
	Flat_Accuracy: 0.9498404755187663
	Flat_F1_Score: 0.6162452968455749
	Flat_Precision: 0.7406090322829657
	Flat_Recall: 0.5289886961436592
Results for: Stem POS BOS
	Flat_Accuracy: 0.9509958062261678
	Flat_F1_Score: 0.6304383299263348
	Flat_Precision: 0.7403506217363869
	Flat_Recall: 0.5502953544161597
Results for: Stem POS PROP BOS
	Flat_Accuracy: 0.9520129512537944
	Flat_F1_Score: 0.6415555321490359
	Flat_Precision: 0.7368059723799022
	Flat_Recall: 0.5698045301776119
Results for: Stem POS PROP NCS BOS
	Flat_Accuracy: 0.9720137212384043
	Flat_F1_Score: 0.7788600017220744
	Flat_Precision: 0.8591605185049479
	Flat_Recall: 0.7152766004811129
Results for: Stem POS PROP NCS OCS BOS
	Flat_Accuracy: 0.9721666719171731
	Flat_F1_Score: 0.7816482114219917
	Flat_Precision: 0.8599564901518695
	Flat_Recall: 0.7185161466851903
Results for: Stem POS PROP NCS OCS INF BOS
	Flat_Accuracy: 0.9737891772606252
	Flat_F1_Score: 0.7910719529407627
	Flat_Precision: 0.8518346262471604
	Fl

In [112]:
printResults(results)

Results for: 0_0
	Flat_Accuracy: 0.9494365093714217
	Flat_F1_Score: 0.6037696142025423
	Flat_Precision: 0.7254254831589321
	Flat_Recall: 0.5186180422264876
Results for: 0_1
	Flat_Accuracy: 0.9489871263925006
	Flat_F1_Score: 0.6158544940507452
	Flat_Precision: 0.7436954149209768
	Flat_Recall: 0.5271166601638705
Results for: 0_2
	Flat_Accuracy: 0.9524821845619498
	Flat_F1_Score: 0.6354554235327103
	Flat_Precision: 0.7490191963404902
	Flat_Recall: 0.5526015951386252
Results for: 0_3
	Flat_Accuracy: 0.9508005482807897
	Flat_F1_Score: 0.6279105407137288
	Flat_Precision: 0.7438043824547222
	Flat_Recall: 0.5444364161849711
Results for: 0_4
	Flat_Accuracy: 0.9474960089871697
	Flat_F1_Score: 0.5982364117281479
	Flat_Precision: 0.7411006845397072
	Flat_Recall: 0.5021707670043415
Results for: 1_0
	Flat_Accuracy: 0.9510335683722051
	Flat_F1_Score: 0.6251911910093216
	Flat_Precision: 0.7277600473512819
	Flat_Recall: 0.5504798464491363
Results for: 1_1
	Flat_Accuracy: 0.9497897203762561
	Flat_F1_Sco

# What Model Learns?

In [115]:
maData = []
i = 0
for line in maLines:
    lineNum, word, ma = maParser(line)
    if lineNum > len(maData):
        maData.append( [line] )
    else:
        maData[lineNum-1].append(line)

X = []
y = []

prevLine = 1
index = 0
for line in maData:
    temp_X = []
    temp_y = []
    BOS = True
    for word in line:
        lineNum, word, ma = maParser(word)
        temp_X.append( mapFeatures(word, ma, BOS, 6) )
        temp_y.append( getLabel(rawData[lineNum-1], word))
        BOS = False
    X.append(temp_X)
    y.append(temp_y)

crf = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=0.1,
            c2=0.1,
            max_iterations=100,
            all_possible_transitions=True, )
        
crf.fit(X, y)

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])



Top likely transitions:
I-ORGANIZATION -> I-ORGANIZATION 4.347910
B-ORGANIZATION -> I-ORGANIZATION 4.002563
I-LOCATION -> I-LOCATION 3.874740
B-PERSON -> I-PERSON 3.843507
B-LOCATION -> I-LOCATION 2.428487
I-PERSON -> I-PERSON 2.182171
I-ORGANIZATION -> O       1.858638
O      -> O       1.353891
B-ORGANIZATION -> O       1.298995
B-ORGANIZATION -> B-LOCATION 0.871935
I-LOCATION -> O       0.771517
B-PERSON -> O       0.768013
O      -> B-PERSON 0.733506
I-PERSON -> O       0.606989
O      -> B-LOCATION 0.377046
I-LOCATION -> B-LOCATION 0.187731
B-LOCATION -> O       0.155373
B-ORGANIZATION -> I-LOCATION 0.086881
O      -> B-ORGANIZATION -0.054770
I-ORGANIZATION -> B-ORGANIZATION -0.089100

Top unlikely transitions:
I-PERSON -> B-ORGANIZATION -0.970206
B-ORGANIZATION -> B-ORGANIZATION -0.989085
B-PERSON -> B-LOCATION -1.034096
B-ORGANIZATION -> I-PERSON -1.192168
I-LOCATION -> B-PERSON -1.387936
I-LOCATION -> B-ORGANIZATION -1.456096
I-PERSON -> B-PERSON -1.821215
I-ORGANIZATION -> I-P

In [117]:
countDic = {
    "O": 0,
    "B-PERSON": 0,
    "I-PERSON": 0,
    "B-ORGANIZATION": 0,
    "I-ORGANIZATION": 0,
    "B-LOCATION": 0,
    "I-LOCATION": 0,
}


for line in y:
    for label in line:
        countDic[label] += 1

In [118]:
countDic

{'B-LOCATION': 2506,
 'B-ORGANIZATION': 2402,
 'B-PERSON': 4568,
 'I-LOCATION': 343,
 'I-ORGANIZATION': 1587,
 'I-PERSON': 1927,
 'O': 152515}

# K-Folds And Run All

In [68]:
path = "/content/drive/MyDrive/2020-2021 401/CS 445/Projects/CS 445 Project 4/NE.ma.txt"

maFile = open(path, "r")
maLines = maFile.read().splitlines()

In [69]:
maData = []
i = 0
for line in maLines:
    lineNum, word, ma = maParser(line)
    if lineNum > len(maData):
        maData.append( [line] )
    else:
        maData[lineNum-1].append(line)

In [70]:
X = []
y = []

prevLine = 1
index = 0
for line in maData:
    temp_X = []
    temp_y = []
    BOS = True
    for word in line:
        lineNum, word, ma = maParser(word)
        temp_X.append( mapFeatures(word, ma, BOS) )
        temp_y.append( getLabel(rawData[lineNum-1], word))
        BOS = False
    X.append(temp_X)
    y.append(temp_y)

In [71]:
X_fold0 = []
X_fold1 = []
X_fold2 = []
X_fold3 = []
X_fold4 = []

y_fold0 = []
y_fold1 = []
y_fold2 = []
y_fold3 = []
y_fold4 = []

In [72]:
X_folds = [ X_fold0, X_fold1, X_fold2, X_fold3, X_fold4 ]

y_folds = [ y_fold0, y_fold1, y_fold2, y_fold3, y_fold4 ]

In [73]:
for index, sentence in enumerate(X):
    X_folds[index % 5].append(sentence)

In [74]:
for index, label in enumerate(y):
    y_folds[index % 5].append(label)

In [ ]:
print(len(X_folds), len(X_folds[0]), len(y_folds), len(y_folds[0]))

In [ ]:
results = {}

for testFold in range(4):
    X_train = []
    y_train = []

    for index, fold in enumerate(X_folds):
        if index != testFold:
            X_train += fold

    for index, labels in enumerate(y_folds):
        if index != testFold:
            y_train += labels

    X_test = X_folds[testFold]
    y_test = y_folds[testFold]

    # print( len(X_train), len(y_train), len(X_test), len(y_test))

    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True, )
    
    crf.fit(X_train, y_train)
    
    labels = list(crf.classes_)
    sorted_labels = sorted( labels, key=lambda name: (name[1:], name[0]))

    predictions = crf.predict(X_test)

    results[testFold] = [   metrics.flat_accuracy_score(y_test, predictions),
                            metrics.flat_f1_score(y_test, predictions, average="weighted"),
                            metrics.flat_precision_score(y_test, predictions, average="weighted"),
                            metrics.flat_recall_score(y_test, predictions, average="weighted")
                        
                        ]

    print( "TestFold is:", testFold ," --> Flat f1 score:", metrics.flat_f1_score(y_test, predictions, average="weighted") )

    print( metrics.flat_classification_report( y_test, predictions, labels=sorted_labels, digits=3) )


In [ ]:
results

In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

# RandomizedSearchCV

In [ ]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.01922045471930707,
    c2=0.03690599493041812,
    max_iterations=100,
    all_possible_transitions=True, )

crf.fit(X_train, y_train)

labels = list(crf.classes_)
sorted_labels = sorted( labels, key=lambda name: (name[1:], name[0]))

predictions = crf.predict(X_test)

print( "Flat f1 score:", metrics.flat_f1_score(y_test, predictions, average="weighted") )

print( metrics.flat_classification_report( y_test, predictions, labels=sorted_labels, digits=3) )


In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])